In [1]:
import sys
sys.path.append("..")
import numpy as np
from project.ship import Ship
from project.tests import Tests
from project.botma import TMA
import time

In [2]:
# Класс для сохранения результатов
tests = Tests('test1')

# Создаем наблюдатель
observer_x, observer_y, observer_course, observer_velocity = 0, 0, 0, 5
observer = Ship('Наблюдатель', observer_x, observer_y, observer_course,
                observer_velocity, verbose=True)

In [3]:
# Моделирование траектории наблюдателя
observer.forward_movement(3 * 60)
observer.change_course(270, 'left', omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, 'right', omega=0.5)
observer.forward_movement(3 * 60)

Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с


In [4]:
def target_func(seed=None):
    np.random.seed(seed)
    b = 0
    d = 10
    c = np.random.uniform(0, 180)
    v = np.random.uniform(3, 15)
    return [b, d, c, v]
p0 = [0., 25., 90., 7.]
std_arr = [0., 0.1, 0.2, 0.3, 0.5, 1.]

In [5]:
print('_______________________________________________________')
for std in std_arr:
    tma = TMA(observer, sd=np.radians(std), tau=2)
    tma.print_verbose()
    print('ММП v6')
    start_time = time.perf_counter()
    dict_results = tma.swarm(n=100, target_func=target_func, p0=p0, seeded=True)
    stop_time = time.perf_counter()
    print('Моделирование 100 результатов закончено за t = {:.1f} с'.format(stop_time - start_time))
    df = tests.get_df(dict_results)
    print('Показатели Монте-Карло:')
    for i in range(4):
        a = df['Точ'].apply(lambda x: x[i])
        print('Вариант {}: Рэф = {:.0f}$, Рэф_мин = {:.0f}'.format(i, 100 * a.mean(), min(a)))
    print('Ка = {:.2f}'.format(df['Ка'].mean()))
    print('Nf = {:.0f}, Nf_max = {:.0f}'.format(df['Nf'].mean(), max(df['Nf'])))
    print('Iter = {:.0f}, Iter_max = {:.0f}'.format(df['Iter'].mean(), max(df['Iter'])))
    print('t = {:.4f}, t_max = {:.4f}'.format(df['t'].mean(), max(df['t'])))
    print('_______________________________________________________')

_______________________________________________________
СКОп = 0.0, tau = 2
ММП v6
Моделирование 100 результатов закончено за t = 2.6 с
Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.00
Nf = 15, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0032, t_max = 0.0066
_______________________________________________________
СКОп = 0.1, tau = 2
ММП v6
Моделирование 100 результатов закончено за t = 2.7 с
Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.01
Nf = 25, Nf_max = 31
Iter = 18, Iter_max = 23
t = 0.0047, t_max = 0.0092
_______________________________________________________
СКОп = 0.2, tau = 2
ММП v6
Моделирование 100 результатов закончено за t = 2.7 с
Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вари

In [6]:
print('_______________________________________________________')
for std in std_arr:
    tma = TMA(observer, sd=np.radians(std), tau=2)
    tma.print_verbose()
    algorithm_name = 'Метод N пеленгов'
    print(algorithm_name)
    start_time = time.perf_counter()
    dict_results = tma.swarm(n=100, algorithm_name=algorithm_name, target_func=target_func, p0=p0, seeded=True)
    stop_time = time.perf_counter()
    print('Моделирование 100 результатов закончено за t = {:.1f} с'.format(stop_time - start_time))
    df = tests.get_df(dict_results)
    print('Показатели Монте-Карло:')
    for i in range(4):
        a = df['Точ'].apply(lambda x: x[i])
        print('Вариант {}: Рэф = {:.0f}$, Рэф_мин = {:.0f}'.format(i, 100 * a.mean(), min(a)))
    print('Ка = {:.2f}'.format(df['Ка'].mean()))
    print('t = {:.4f}, t_max = {:.4f}'.format(df['t'].mean(), max(df['t'])))
    print('_______________________________________________________')

_______________________________________________________
СКОп = 0.0, tau = 2
Метод N пеленгов
Моделирование 100 результатов закончено за t = 2.4 с
Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.00
t = 0.0003, t_max = 0.0007
_______________________________________________________
СКОп = 0.1, tau = 2
Метод N пеленгов
Моделирование 100 результатов закончено за t = 2.3 с
Показатели Монте-Карло:
Вариант 0: Рэф = 98$, Рэф_мин = 0
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.01
t = 0.0003, t_max = 0.0005
_______________________________________________________
СКОп = 0.2, tau = 2
Метод N пеленгов
Моделирование 100 результатов закончено за t = 2.3 с
Показатели Монте-Карло:
Вариант 0: Рэф = 81$, Рэф_мин = 0
Вариант 1: Рэф = 98$, Рэф_мин = 0
Вариант 2: Рэф = 98$, Рэф_мин = 0
Вариант 3: Рэф = 99$, Рэф_мин = 0
К